https://colab.research.google.com/drive/1riwAm3M-YeYreFbRTYhw0BwZs9Nff5Pg

In [0]:
!wget http://www.gutenberg.org/files/11/11-0.txt

--2019-10-02 09:06:08--  http://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173595 (170K) [text/plain]
Saving to: ‘11-0.txt’

11-0.txt            100%[===================>] 169.53K   357KB/s    in 0.5s    

2019-10-02 09:06:09 (357 KB/s) - ‘11-0.txt’ saved [173595/173595]



In [0]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers.recurrent import SimpleRNN
import numpy as np

Using TensorFlow backend.


In [0]:
fin=open('11-0.txt',encoding='utf-8-sig')
lines=[]
for line in fin:
  line = line.strip().lower()
  #line = line.decode("ascii","ignore")
  if(len(line)==0):
    continue
  lines.append(line)
fin.close()
text = " ".join(lines)

In [0]:
text[52:100]

', by lewis carroll this ebook is for the use of '

In [0]:
import re
text = text.lower()
text = re.sub('[^0-9a-zA-Z]+',' ',text)

In [0]:
from collections import Counter
counts = Counter()
counts.update(text.split())
words = sorted(counts, key=counts.get, reverse=True)
nb_words = len(text.split())
word2index = {word: i for i, word in enumerate(words)}
index2word = {i: word for i, word in enumerate(words)}

In [0]:
SEQLEN = 10
STEP = 1
input_words = []
label_words = []
text2=text.split()
for i in range(0,nb_words-SEQLEN,STEP):
    x=text2[i:(i+SEQLEN)]
    y=text2[i+SEQLEN]
    input_words.append(x)
    label_words.append(y)
print('input words list: ','\n',input_words[0])
print('label(output) words list: ','\n',label_words[0])

input words list:  
 ['project', 'gutenberg', 's', 'alice', 's', 'adventures', 'in', 'wonderland', 'by', 'lewis']
label(output) words list:  
 carroll


In [0]:
total_words = len(set(words))
X = np.zeros((len(input_words), SEQLEN, total_words), dtype=np.bool)
y = np.zeros((len(input_words), total_words), dtype=np.bool)

In [0]:
for i, input_word in enumerate(input_words):
     for j, word in enumerate(input_word):
         X[i, j, word2index[word]] = 1
     y[i,word2index[label_words[i]]]=1
print('Shape of X: ',X.shape)
print('Shape of y: ',y.shape)

Shape of X:  (30527, 10, 3043)
Shape of y:  (30527, 3043)


In [0]:
HIDDEN_SIZE = 128
BATCH_SIZE = 32
NUM_ITERATIONS = 100
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [0]:
from keras.layers import LSTM

In [0]:
model = Sequential()
model.add(LSTM(HIDDEN_SIZE,return_sequences=False,input_shape=(SEQLEN,total_words)))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               1624064   
_________________________________________________________________
dense_1 (Dense)              (None, 3043)              392547    
Total params: 2,016,611
Trainable params: 2,016,611
Non-trainable params: 0
_________________________________________________________________


In [0]:
int(len(input_words)*0.1)

3052

In [0]:
len(input_words)

30527

In [0]:
for iteration in range(50):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION, validation_split = 0.1)
    test_idx = np.random.randint(int(len(input_words)*0.1)) * (-1)
    test_words = input_words[test_idx]
    print("Generating from seed: %s" % (test_words))
    for i in range(NUM_PREDS_PER_EPOCH):        
        Xtest = np.zeros((1, SEQLEN, total_words))
        for i, ch in enumerate(test_words):
            Xtest[0, i, word2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred,end=' ')
        test_words = test_words[1:] + [ypred]

In [0]:
from keras.layers import LSTM, Input, Bidirectional

In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(HIDDEN_SIZE,return_sequences=False),input_shape=(SEQLEN,total_words)))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 256)               3248128   
_________________________________________________________________
dense_2 (Dense)              (None, 3043)              782051    
Total params: 4,030,179
Trainable params: 4,030,179
Non-trainable params: 0
_________________________________________________________________


In [0]:
for iteration in range(50):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION, validation_split = 0.1)
    test_idx = np.random.randint(int(len(input_words)*0.1)) * (-1)
    test_words = input_words[test_idx]
    print("Generating from seed: %s" % (test_words))
    for i in range(NUM_PREDS_PER_EPOCH):        
        Xtest = np.zeros((1, SEQLEN, total_words))
        for i, ch in enumerate(test_words):
            Xtest[0, i, word2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred,end=' ')
        test_words = test_words[1:] + [ypred]

Iteration #: 0
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 38s 1ms/step - loss: 6.1547 - val_loss: 8.0670
Generating from seed: ['work', 'by', 'people', 'who', 'agree', 'to', 'be', 'bound', 'by', 'the']
little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little little little little of the little little little little ==================================================
Iteration #: 1
Train on 27474 samples, validate on 3053 sampl